In [ ]:
import os
from typing import List, Dict, Literal
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from openai import OpenAI
import json

# Load API key from .env file
load_dotenv()
client = OpenAI(
    base_url = "https://openai.vocareum.com/v1",
    api_key=os.getenv("OPENAI_API_KEY"))

class FitnessUser:
    """Represents a fitness app user."""
    def __init__(self, id: str, age: int, fitness_level: int, 
                 goals: List[str], preferences: List[str], 
                 limitations: List[str] = None):
        self.id = id
        self.age = age
        self.fitness_level = fitness_level
        self.goals = goals
        self.preferences = preferences
        self.limitations = limitations or []

    def __str__(self):
        return f"User {self.id}: Level {self.fitness_level}, Goals: {', '.join(self.goals)}"

class DailyWorkout(BaseModel):
    type: str = Field(..., example="strength training")
    duration: int = Field(..., description="Duration in minutes", example=45)
    intensity: Literal["low", "moderate", "high"] = Field(..., example="moderate")
    description: str = Field(..., example="Full-body strength training with dumbbells.")

class WeeklySchedule(BaseModel):
    Monday: DailyWorkout
    Tuesday: DailyWorkout
    Wednesday: DailyWorkout
    Thursday: DailyWorkout
    Friday: DailyWorkout
    Saturday: DailyWorkout
    Sunday: DailyWorkout

class LLMWorkoutPlan(BaseModel):
    reasoning: str = Field(..., example="Plan balances cardio and strength with time limits respected.")
    weekly_schedule: WeeklySchedule
    considerations: str = Field(..., example="Used home workouts and capped sessions at 30 min.")

# ======== TODO: AGENT 1 — Deterministic Planner ========
# Create a rule-based planner that adjusts:
# - number of workout days
# - intensity
# - workout types
# - session duration
# based on fitness level and goals

def deterministic_agent(user: FitnessUser) -> Dict:
    """
    Implement your logic here to generate:
    {
        "weekly_schedule": {
            "Monday": {"type": "strength training", "duration": 45, "intensity": "moderate", "description": "..."},
            ...
        }
    }
    """
    # Your code goes here
    fitness_level = user.fitness_level
    goals = set(goal.lower() for goal in user.goals)
    limitations = set(lim.lower() for lim in user.limitations)

    # Defaults
    base_duration = {1: 20, 2: 30, 3: 40, 4: 50, 5: 60}
    intensity_level = {1: "low", 2: "low", 3: "moderate", 4: "moderate", 5: "high"}
    num_days = {1: 3, 2: 4, 3: 5, 4: 6, 5: 6}

    duration = base_duration[fitness_level]
    intensity = intensity_level[fitness_level]
    days = num_days[fitness_level]

    # Mapping for goal → workout type
    goal_to_type = {
        "weight management": "cardio",
        "stress reduction": "yoga",
        "strength building": "strength training",
        "joint mobility": "mobility exercises",
        "endurance": "interval training",
    }

    # Final weekly schedule
    schedule = {}
    weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

    # Determine prioritized types based on goals
    workout_types = [goal_to_type[goal] for goal in goals if goal in goal_to_type]

    # Fallback type if none matched
    if not workout_types:
        workout_types = ["full-body workout"]

    # Remove types incompatible with limitations
    if "joint stiffness" in limitations:
        workout_types = [t for t in workout_types if t not in ["running", "plyometrics"]]
        if "mobility exercises" not in workout_types:
            workout_types.append("mobility exercises")
    if "limited equipment" in limitations:
        workout_types = [t for t in workout_types if t != "strength training"]
        if "bodyweight training" not in workout_types:
            workout_types.append("bodyweight training")

    # Assign workouts to days
    active_days = weekdays[:days]
    for i, day in enumerate(weekdays):
        if day in active_days:
            workout_type = workout_types[i % len(workout_types)]
            schedule[day] = {
                "type": workout_type,
                "duration": min(duration, 30) if "time constraints" in limitations else duration,
                "intensity": intensity,
                "description": f"{workout_type.title()} focused session tailored for your goals."
            }
        else:
            schedule[day] = {
                "type": "rest",
                "duration": 0,
                "intensity": "none",
                "description": "Rest day for recovery."
            }

    return {
        "reasoning": f"Planned based on fitness level {fitness_level}, goals {user.goals}, and limitations {user.limitations}",
        "weekly_schedule": schedule,
        "considerations": "Deterministic logic applied. No personalization from LLM."
    }


# ======== AGENT 2 — LLM-Based Planner ========
# We've handled the API part. Your task is to COMPLETE THE PROMPT below
# that will instruct the LLM how to generate the plan.

def llm_agent(user: FitnessUser) -> Dict:
    goals_text = ", ".join(user.goals)
    preferences_text = ", ".join(user.preferences)
    limitations_text = ", ".join(user.limitations) if user.limitations else "None"

    prompt = f"""
    As a certified fitness trainer, create a personalized weekly workout plan for this client.
    
    Client Information:
    - Age: {user.age}
    - Fitness Level: {user.fitness_level}/5
    - Goals: {goals_text}
    - Preferences: {preferences_text}
    - Limitations: {limitations_text}
    
    Instructions:
    - Tailor the plan to the user's fitness level and goals.
    - Respect any limitations (e.g., equipment, joint issues, time constraints).
    - Incorporate preferences (e.g., home workouts, swimming) where appropriate.
    - Distribute workouts across the week, including rest days.
    - Each active day must include:
        - type: (e.g., "cardio", "yoga", "strength training")
        - duration: (integer, minutes)
        - intensity: ("low", "moderate", or "high")
        - description: (short summary)
    
    Format:
    Respond in raw JSON only using this schema:
    
    {LLMWorkoutPlan.model_json_schema()}

    Example (partial response for inspiration only):

    {{
      "reasoning": "Client prefers home workouts and has time constraints...",
      "weekly_schedule": {{
        "Monday": {{
          "type": "strength training",
          "intensity": "moderate",
          "duration": 30,
          "description": "Full-body circuit using bodyweight and resistance bands."
        }},
        ...
      }},
      "considerations": "Avoided high-impact routines..."
    }}
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a certified fitness trainer."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.2,
        )
        result_text = response.choices[0].message.content
        return json.loads(result_text)

    except Exception as e:
        fallback = deterministic_agent(user)
        return {
            "reasoning": f"LLM planning failed: {str(e)}",
            "weekly_schedule": fallback["weekly_schedule"],
            "considerations": "Fallback to rule-based plan."
        }


# ======== COMPARISON LOGIC (DO NOT EDIT) ========

def compare_workout_planning(users: List[FitnessUser]):
    print("\n===== WORKOUT PLAN COMPARISON =====")
    for i, user in enumerate(users, 1):
        print(f"\n--- User {i}: {user.id} ---")
        print(f"Age: {user.age} | Fitness Level: {user.fitness_level}/5")
        print(f"Goals: {', '.join(user.goals)}")
        print(f"Preferences: {', '.join(user.preferences)}")
        print(f"Limitations: {', '.join(user.limitations)}")

        det_plan = deterministic_agent(user)
        print("\n[Deterministic Agent]")
        for day, workout in det_plan["weekly_schedule"].items():
            print(f"- {day}: {workout['type']} ({workout['intensity']}, {workout['duration']} min)")

        llm_plan = llm_agent(user)
        print("\n[LLM Agent]")
        print(f"Reasoning: {llm_plan.get('reasoning', 'No reasoning provided')}")
        for day, workout in llm_plan["weekly_schedule"].items():
            print(f"- {day}: {workout['type']} ({workout['intensity']}, {workout['duration']} min)")
            print(f"  {workout['description']}")
        print(f"Considerations: {llm_plan.get('considerations', 'None')}")


# ======== SAMPLE USERS ========

def main():
    users = [
        FitnessUser(
            id="U001",
            age=35,
            fitness_level=2,
            goals=["weight management", "stress reduction"],
            preferences=["home workouts", "morning routines"],
            limitations=["limited equipment", "time constraints (max 30 min/day)"]
        ),
        FitnessUser(
            id="U002",
            age=55,
            fitness_level=3,
            goals=["joint mobility", "strength building"],
            preferences=["outdoor activities", "swimming"],
            limitations=["mild joint stiffness"]
        )
    ]

    compare_workout_planning(users)

if __name__ == "__main__":
    main()



===== WORKOUT PLAN COMPARISON =====

--- User 1: U001 ---
Age: 35 | Fitness Level: 2/5
Goals: weight management, stress reduction
Preferences: home workouts, morning routines
Limitations: limited equipment, time constraints (max 30 min/day)

[Deterministic Agent]
- Monday: cardio (low, 30 min)
- Tuesday: yoga (low, 30 min)
- Wednesday: bodyweight training (low, 30 min)
- Thursday: cardio (low, 30 min)
- Friday: rest (none, 0 min)
- Saturday: rest (none, 0 min)
- Sunday: rest (none, 0 min)
